In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Github/3DQSAR

Mounted at /content/drive
/content/drive/My Drive/Github/3DQSAR


In [2]:
# !pip install --upgrade batchgenerators

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import BatchNormalization,Flatten,\
Add,Input,Dense, Dropout, Activation, InputLayer, Concatenate
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from scipy.ndimage import gaussian_filter
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os 
import pickle
import scipy
import random

In [4]:
def categorical_label(Y):
  Y_cat=np.zeros(Y.shape[0])
  for idx in range(len(Y)):
    if Y[idx] >=6:
      Y_cat[idx]=1
    else:
      Y_cat[idx]=0
  return Y_cat

In [10]:
def split_shuffle(X_1,X_2,Y,augment,aug_iter):
    ###Reshape
    Y=np.asarray(Y)
    
    X_1 = np.reshape(X_1,(X_1.shape[0],X_1.shape[1],X_1.shape[2],X_1.shape[3],1))
    X_2 = np.reshape(X_2,(X_2.shape[0],X_2.shape[1],X_2.shape[2],X_2.shape[3],1))
    
    mol1 = np.where(Y==(5.154902))
    mol2 = np.where(Y==(5.815309))
    mol3 = np.where(Y==(6.69897))
    mol4 = np.where(Y==(7.85))
    mol5 = np.where(Y==(8.102373)) 

    test_index = np.hstack([mol1,mol2,mol3,mol4,mol5])

    test_index = np.squeeze(test_index)

    X_1_train = np.delete(X_1,test_index,0)
    X_2_train = np.delete(X_2,test_index,0)
    X_1_test = X_1[test_index]
    X_2_test = X_2[test_index]

    Y_train = np.delete(Y,test_index)
    Y_test = Y[test_index]
    del X_1,X_2,Y
           
    mol1 = np.where(Y_train==(5.19382))
    mol2 = np.where(Y_train==(5.497573))
    mol3 = np.where(Y_train==(5.699622))
    mol4 = np.where(Y_train==(5.9))
    mol5 = np.where(Y_train==(6.346787))

    mol6 = np.where(Y_train==(6.102373))
    mol7 = np.where(Y_train==(6.63))
    mol8 = np.where(Y_train==(7.1))
    mol9 = np.where(Y_train==(7.9))
    mol10 = np.where(Y_train==(8.)) 
    
    val_index = np.hstack([mol1,mol2,mol3,mol4,mol5,
                           mol6,mol7,mol8,mol9,mol10])
    
    val_index = np.squeeze(val_index)

    X_1_val = X_1_train[val_index]
    X_2_val = X_2_train[val_index]
    X_1_train = np.delete(X_1_train, val_index, 0)
    X_2_train = np.delete(X_2_train, val_index, 0)

    Y_val = Y_train[val_index]
    Y_train = np.delete(Y_train, val_index)

    
    permutation_train = np.random.RandomState(seed=42)\
                       .permutation(X_1_train.shape[0])
    permutation_test = np.random.RandomState(seed=42)\
                       .permutation(X_1_test.shape[0])
    permutation_val = np.random.RandomState(seed=42)\
                       .permutation(X_1_val.shape[0])

    split_train = int(np.around(X_1_train.shape[0]/4,0))
    split_val = int(np.around(X_1_val.shape[0]/4,0))
    split_test = int(np.around(X_1_test.shape[0]/4,0))

    X_1_val =   X_1_val[permutation_val]#[:split_val] 
    X_2_val =   X_2_val[permutation_val]#[:split_val] 
    X_1_train = X_1_train[permutation_train]#[:split_train]
    X_2_train = X_2_train[permutation_train]#[:split_train]
    Y_val =     Y_val[permutation_val]#[:split_val]
    Y_train =   Y_train[permutation_train]#[:split_train] 
    X_1_test =  X_1_test[permutation_test]#[:split_test]
    X_2_test =  X_2_test[permutation_test]#[:split_test]
    Y_test =    Y_test[permutation_test]#[:split_test]

    Y_train = categorical_label(Y_train)
    Y_test = categorical_label(Y_test)
    Y_val = categorical_label(Y_val)

    if augment and aug_iter!=0:
      indexes = np.where(Y_train>7.25)
      for _ in np.arange(aug_iter):
        pass
        # new_X = scipy.ndimage.gaussian_filter(X_1_train[indexes],sigma=(1))
        # X_1_train = np.concatenate([X_1_train,new_X])

        # new_X = scipy.ndimage.gaussian_filter(X_2_train[indexes],sigma=(1))
        # X_2_train = np.concatenate([X_2_train,new_X])

        # Y_train = np.concatenate([Y_train,Y_train[indexes]])

        # new_X = scipy.ndimage.rotate(axes=(1,2),X_1_train[indexes],random.choice([90,180,270]),reshape=False,order=3)
        # X_1_train = np.concatenate([X_1_train,new_X])

        # new_X = scipy.ndimage.rotate(axes=(1,2),X_2_train[indexes],random.choice([90,180,270]),reshape=False,order=3)
        # X_2_train = np.concatenate([X_2_train,new_X])

        # Y_train = np.concatenate([Y_train,Y_train[indexes]])

    return X_1_train, X_1_test, X_1_val, X_2_train, X_2_test, X_2_val, Y_train, Y_test, Y_val

In [12]:
#OVER SAMPLING
def resampling(x1,x2, y, number):
  number = number*1001

  hit = np.where(y == 1.0)
  miss = np.where(y == 0.0)
  print(hit)
  print(miss)
  X_1_hit = x1[hit]
  X_2_hit = x2[hit]
  X_1_miss = x1[miss]
  X_2_miss = x2[miss]
  Y_hit = y[hit]
  Y_miss = y[miss]

  permutation_hit = np.random.RandomState(seed=42).permutation(X_1_hit.shape[0])
  permutation_miss = np.random.RandomState(seed=42).permutation(X_1_miss.shape[0])

  X_1_hit = X_1_hit[permutation_hit]
  X_2_hit = X_2_hit[permutation_hit]
  X_1_miss = X_1_miss[permutation_miss]
  X_2_miss = X_2_miss[permutation_miss]
  Y_hit = Y_hit[permutation_hit]
  Y_miss = Y_miss[permutation_miss]

  X_1_copy = X_1_hit[:number].copy()
  X_2_copy = X_2_hit[:number].copy()
  Y_copy = Y_hit[:number].copy()
  X_1_resampled = np.concatenate((X_1_hit, X_1_copy, X_1_miss))
  X_2_resampled = np.concatenate((X_2_hit, X_2_copy, X_2_miss))
  Y_resampled = np.concatenate((Y_hit, Y_copy, Y_miss))
  
  return X_1_resampled, X_2_resampled, Y_resampled

# X_1_train, X_2_train, Y_train = resampling(X_1_train, X_2_train, Y_train, 6)

In [13]:
def build_CNN(X_1_train):
    model = Sequential(name='CNN')

    tf.random.set_seed(42)
    np.random.seed(42)
    inputs = Input(shape=(X_1_train.shape[1], X_1_train.shape[2],X_1_train.shape[3],X_1_train.shape[4]))
    model=inputs
    model=BatchNormalization()(model)
    model = Conv3D(32, (2, 2, 2), strides=(1, 1, 1), activation='relu')(model)
    model = MaxPooling3D(pool_size=(2,2,2))(model)
    model = Conv3D(64, (2, 2, 2), strides=(1, 1, 1), activation='relu')(model)
    model = MaxPooling3D(pool_size=(2,2,2))(model)
    model = Conv3D(128, (2, 2, 2), strides=(1, 1, 1), activation='relu')(model)
    model = MaxPooling3D(pool_size=(2,2,2))(model)
    model = Flatten()(model)
    model = Dense(100,activation='relu')(model)
    output = model

    model = Model(inputs=inputs, outputs=output)
    
    return model

In [14]:
def two_headed_network(X_1_train):
    tf.random.set_seed(42)
    np.random.seed(42)
    X_1_CNN = build_CNN(X_1_train)
    X_2_CNN = build_CNN(X_1_train)
    network = Add()([X_1_CNN.output,X_2_CNN.output])
    network = Dense(100,activation='relu')(network)
    network = Dense(50,activation='relu')(network)
    network = Dense(10,activation='relu')(network)
    network = Dense(1,activation = 'sigmoid')(network)
    model = Model([X_1_CNN.input, X_2_CNN.input], network)
    optimizer = Adam(learning_rate=1e-3)
    model.compile(loss='binary_crossentropy', optimizer=optimizer)
    
    return model  
# CNN = two_headed_network(X_1_train,None)

In [15]:
def train(CNN,X_1_train, X_1_val, X_2_train,\
  X_2_val, Y_train, Y_val):

  monitor = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=30,
                          verbose=1, 
                          mode='auto',
                          restore_best_weights=True)
  tf.random.set_seed(42)
  np.random.seed(42)
  CNN.fit([X_1_train, X_2_train],
          Y_train,
          batch_size=32,
          epochs=1000, 
          validation_data=([X_1_val, X_2_val],
                            Y_val),
                            callbacks=[monitor],
          shuffle=True,
          verbose=1)
  
  model_loss = CNN.evaluate([X_1_val,X_2_val], Y_val, verbose=0)
  return model_loss,CNN

In [16]:
def run(itr):
  tf.random.set_seed(42)
  np.random.seed(42)
  X_1,X_2,Y = pickle.load(open("/content/drive/My Drive/Github/resized_first_1000_data_full.p", "rb"))
  # X_1,X_2,Y = pickle.load(open("/content/drive/My Drive/Github/resized_1_10_data_full.p", "rb"))

  Y = np.asarray(Y)
  X_1_train, X_1_test, X_1_val, X_2_train, X_2_test,\
  X_2_val, Y_train, Y_test, Y_val = split_shuffle(X_1,X_2,Y,augment=False,aug_iter=itr)
  del X_1,X_2,Y
  CNN = two_headed_network(X_1_train)
  loss,CNN = train(CNN,X_1_train, X_1_val, X_2_train,\
                 X_2_val, Y_train, Y_val)
  plt.hist(Y_train)
  return loss,CNN

In [17]:
loss,CNN = run(0)
tf.keras.backend.clear_session()

Epoch 1/1000
 361/1627 [=====>........................] - ETA: 17s - loss: 0.4798

KeyboardInterrupt: ignored